In [30]:
%matplotlib inline
import numpy
import pysrt
import gensim
import textacy
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [127]:
# Load transcripts and preprocess the text
transcripts = []
transcript_single = []

for line in open('../data/test_video_list.txt'):
    srt_name = line.replace('tvnews', '../data').replace('mp4', 'cc5.srt')[:-1]
    offsets = []
    transcript = ''
    replace_list = ['\n', '>', '.', ',', '?', '!', '\'', '"', '-', '(', ')']
    #subs = pysrt.open(srt_name)
    subs = pysrt.open('../data/videos/MSNBC_20110830_080000_The_Rachel_Maddow_Show.cc5.srt')
    idx = 0
    for sub in subs:
        offsets.append((len(transcript), sub.start, sub.end))
        text = sub.text
        for token in replace_list:
            text = text.replace(token, ' ')
        transcript += text.upper() + ' '
        if idx >= 10:
            idx = 0;    
            transcript_single.append(transcript)
            transcript = ''
        else:
            idx += 1
    transcripts.append((None, transcript, offsets))
    break

In [128]:
# Extract PROPN and NOUN using textacy
# docs = [textacy.doc.Doc(transcript, lang=u'en', metadata={'offsets': offsets}) for (video, transcript, offsets) in transcripts]
# corpus = textacy.Corpus(u'en', docs=docs)

docs = [textacy.doc.Doc(transcript, lang=u'en') for transcript in transcript_single]
corpus = textacy.Corpus(u'en', docs=docs)

transcript_tokens = [
    list(textacy.extract.words(doc, filter_nums=True, include_pos=['PROPN', 'NOUN'])) 
    for doc in corpus.docs]


In [ ]:
# Build dictionary and corpora using gensim
transcript_tokens = [[str(token) for token in tokens if len(str(token)) > 1] for tokens in transcript_tokens]
print(len(transcript_tokens))
dictionary = gensim.corpora.Dictionary(transcript_tokens)
print(dictionary)

corpus = [dictionary.doc2bow(tokens) for tokens in transcript_tokens]
#print(corpus)

In [ ]:
# Topic extraction
# step 1 -- initialize a model
tfidf = gensim.models.TfidfModel(corpus) 

In [131]:
# step 2 -- apply the transformation to a whole corpus
corpus_tfidf = tfidf[corpus]
#for doc in corpus_tfidf:
#    print(doc)

In [ ]:
# step 3 -- # initialize an LSI transformation
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=20) 
corpus_lsi = lsi[corpus_tfidf]

lsi.print_topics(20)

#for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
#    print(doc)

In [ ]:
# step 3 -- # initialize an Random Projection transformation
#rp = gensim.models.RpModel(corpus_tfidf, id2word=dictionary, num_topics=100) 
#corpus_rp = rp[corpus_tfidf]

lda = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20) 
corpus_lda = lda[corpus_tfidf]


# hdp = gensim.models.HdpModel(corpus_tfidf, id2word=dictionary) 
# corpus_hdp = hdp[corpus_tfidf]

lda.print_topics(20)